# Create Bot V1 using AWS SDK Python

Using AWS Lex V1 APIs, we can create a initial Lex bot from a utterances, intent file -sampleutterances.csv

We will use a template file easily available from AWS website - LexIntent_Template.json

We would get a output file that can also be manually imported- LexbotImport.zip

Author: msb.nisha@gmail.com

In [1]:
import pandas as pd
import json
import zipfile
import boto3
from botocore.config import Config

In [2]:
#Define Bot Name
bot_name = 'mylexbot'

In [3]:
#Setting up config settings for AWS
my_config = Config(
    region_name = 'us-east-1',
    retries = {
        'max_attempts': 20,
        'mode': 'standard'
    }
)

In [4]:
#Read the sample utterances files
dataset = pd.read_csv("sampleutterances.csv")

In [5]:
dataset.head()

,intent,utterances
0,orderfood,I need 2 buckets of fried chicken
1,orderfood,Can i have cup of coffee to go
2,orderfood,Coffee please
3,orderfood,I would like 2 cucumber sandwiches
4,requestcheck,Can I have the check please


In [6]:
#Intents in the sampleset
intents_org = dataset['intent'].unique()
print("Intents are :", intents_org)

Intents are : ['orderfood' 'requestcheck' 'cancelorder']


In [7]:
#Reading the Intent Template file
with open("LexIntent_Template.json", 'r+') as fh1:
    data = json.load(fh1)
    data_int = data['resource']
fh1.close()

In [8]:
#Creating a Lex bot Template file
outputfile = 'MyLexBot_Import.json'

with open("LexBot_Template.json", 'r+') as fh2:
    data = json.load(fh2)
    data['resource']['name'] = bot_name 
    intent_defs = []
    
    for intentname in intents_org:
        data_int['name'] = intentname       
        data_int['sampleUtterances'] = dataset[dataset['intent']== intentname]['utterances'].values.tolist()
        intent_defs.append(data_int.copy())
    data['resource']['intents'] = intent_defs
fh2.close()

with open (outputfile, 'w+') as fh3:
    json.dump(data, fh3)
fh3.close()

In [9]:
#Zip the Json file
zipfile.ZipFile('LexbotImport.zip', mode='w').write(outputfile)

In [12]:
#Calling Import function to build lex bot using the Json file
client = boto3.client('lex-models', config=my_config)

filename = "LexbotImport.zip"
with open(filename, 'rb') as binary_file:
    binary_file_data = binary_file.read()

response = client.start_import(
    payload=binary_file_data ,
    resourceType='BOT' ,
    mergeStrategy='OVERWRITE_LATEST'    
)